In [ ]:
%matplotlib inline

from __future__ import division
import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from scipy.signal import convolve2d
    
from menpo.image import Image

#Load Images 

In [ ]:
import menpo.io as mio
from menpo.landmark import labeller, ibug_face_66

images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/trainset/', verbose=True, 
                           max_images=10):
    
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    images.append(i)

In [ ]:
from menpo.visualize import visualize_images

visualize_images(images)

# Padding functions

In [ ]:
from __future__ import division
import numpy as np


def pad_center(pixels, ext_shape, mode='constant'):
    _, h, w = pixels.shape

    h_margin = (ext_shape[0] - h) // 2
    w_margin = (ext_shape[1] - w) // 2

    h_margin2 = h_margin
    if h + 2 * h_margin < ext_shape[0]:
        h_margin += 1

    w_margin2 = w_margin
    if w + 2 * w_margin < ext_shape[1]:
        w_margin += 1

    pad_width = ((0, 0), (h_margin, h_margin2), (w_margin, w_margin2))

    return np.lib.pad(pixels, pad_width, mode=mode)


def crop_center(pixels, shape, f_shape):
    _, h, w = pixels.shape

    h_offset = f_shape[0] // 2
    w_offset = f_shape[1] // 2
    h_margin = (h - shape[0]) // 2
    w_margin = (w - shape[1]) // 2
    
    h_step =  h_margin / h_offset
    w_step =  w_margin / w_offset
    
    h_corrector = 1 if np.remainder(h - shape[0], 2) != 0 else 0
    w_corrector = 1 if np.remainder(w - shape[1], 2) != 0 else 0
    
    print h_corrector
    print w_corrector
        
    return pixels[:, 
                  h_margin + h_corrector:-h_margin, 
                  w_margin + w_corrector:-w_margin]


def pad_bottomright(pixels, shape, mode='constant'):
    _, h, w = pixels.shape

    h_margin = shape[0] - h
    w_margin = shape[1] - w

    pad_width = ((0, 0), (0, h_margin), (0, w_margin))

    return np.lib.pad(pixels, pad_width, mode=mode)


def crop_bottomright(pixels, shape, f_shape):
    _, h, w = pixels.shape

    h_offset = f_shape[0] // 2
    w_offset = f_shape[1] // 2
    h_margin = h - shape[0]
    w_margin = w - shape[1]

    return pixels[:, 
                  h_offset:h_offset-h_margin, 
                  w_offset:w_offset-w_margin]

# Compute convolutions 

In [ ]:
# set options
ind = 0

filter_type = 'landmark'
l = 46

correlation = True

a = 1
mode = 'constant'
normalize = True

In [ ]:
# select image
i = images[ind].copy()


if filter_type is 'random':
    # build random filter
    f = Image(np.random.randn(1, 17, 17))
else:
    # extract filter around landmark
    f = i.extract_patches_around_landmarks(group='ibug_face_66', 
                                           patch_size=(17, 17))[l].copy()
    
if correlation:
    # flip filter if correlation is required
    f = Image(f.pixels[0, ::-1, ::-1])

if normalize:
    # normalize image and filter if necessary
    i.normalize_norm_inplace()
    f.normalize_norm_inplace()

# visualize
i.view()
f.view(new_figure=True)

### Center padded convolution

In [ ]:
# extended shape
ext_h = i.shape[0] + f.shape[0] - 1
ext_w = i.shape[1] + f.shape[1] - 1
ext_shape = (a * ext_h, a * ext_w)

# extend image and filter
ext_i = pad_center(i.pixels, ext_shape, mode=mode)
ext_f = pad_center(f.pixels, ext_shape)

# compute ffts of extended image and extended filter
fft_ext_i = fft2(ext_i)
fft_ext_f = fft2(ext_f)

# compute extended convolution in Fourier domain
fft_ext_c = fft_ext_f * fft_ext_i

# compute ifft of extended convolution
ext_c1 =  np.real(ifftshift(ifft2(fft_ext_c), axes=(-2, -1)))

# crop extended convolution
c1 = crop_center(ext_c1, i.shape, f.shape)

# visualize
Image(ext_i).view()
Image(ext_f).view(new_figure=True)
Image(ext_c1).view(new_figure=True)
Image(c1).view(new_figure=True)

# info
print 'Original images shape is: \t', i.pixels.shape
print 'Original filter shapes is: \t', f.pixels.shape
print 'Extended image shape is: \t', ext_i.shape
print 'Extended filter shape is: \t', ext_f.shape
print 'Extended convolution shape is: \t', ext_c1.shape
print 'Final convolution shapes is: \t', c1.shape

### Bottom-right padded convolution 

In [ ]:
# extended shape
ext_h = i.shape[0] + f.shape[0] - 1
ext_w = i.shape[1] + f.shape[1] - 1
ext_shape = (a * ext_h, a * ext_w)

# extend image and filter
ext_i = pad_bottomright(i.pixels, ext_shape, mode=mode)
ext_f = pad_bottomright(f.pixels, ext_shape)

# compute ffts of extended image and extended filter
fft_ext_i = fft2(ext_i)
fft_ext_f = fft2(ext_f)

# compute extended convolution in Fourier domain
fft_ext_c = fft_ext_f * fft_ext_i

# compute ifft of extended convolution
ext_c2 =  np.real(ifft2(fft_ext_c))

# crop extended convolution
c2 = crop_bottomright(ext_c2, i.shape, f.shape)

# visualize
Image(ext_i).view()
Image(ext_f).view(new_figure=True)
Image(ext_c2).view(new_figure=True)
Image(c2).view(new_figure=True)

# info
print 'Original images shape is: \t', i.pixels.shape
print 'Original filter shapes is: \t', f.pixels.shape
print 'Extended image shape is: \t', ext_i.shape
print 'Extended filter shape is: \t', ext_f.shape
print 'Extended convolution shape is: \t', ext_c2.shape
print 'Final convolution shapes is: \t', c2.shape

### Spatial convolution using scipy

In [ ]:
c3 = convolve2d(i.pixels[0], f.pixels[0], mode='same')

# visualize
i.view()
f.view(new_figure=True)
Image(c3).view(new_figure=True)

# info
print 'Original images shape is: \t', i.pixels.shape
print 'Original filter shapes is: \t', f.pixels.shape
print 'Final convolution shapes is: \t', c2.shape

# Check convolution results

In [ ]:
print 'Center padded convolution is equal to spatial convolution:', np.allclose(c1, c3)

print 'Bottom-right padded convolution is equal to spatial convolution:', np.allclose(c2, c3)

print 'Center padded and bottom-right convolutions are equal:', np.allclose(c1, c2) 